In [2]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob

#import numpy as np
#import loadTidyCSV.py

def loadTidy(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)
tidyDataDir = '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/'
tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(\
    "/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse*.csv"))
print("data files: \n", dataFiles)


print("\n loading and appending to prior pandas data frame")
dataLst = []
for file in dataFiles:
    print(file)
    dataLst.append(loadTidy(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

data files: 
 [ '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse4.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse5.csv']

 loading and appending to prior pandas data frame
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAn

In [3]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'], 
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'], 
      dtype='<U16')}


In [24]:
######### PARAMETERS #########
threshTPs_stdFromMean = 1

In [70]:
### get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 

######### MAIN LOOP ##########
totalNumComparisons = 0;
for (stimA, stimB) in stimCmbTpl:
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    
    #### select data by animals and sessions
    for animal in metaDct['animals']:
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            
            ## get both stim
            df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
            df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
            print(stimA,stimB)
            
            ## get lists of trial numbers of each stimuli's presentations 
            trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
            trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
            print(trials_stimA,trials_stimB)
        
            #### get number of timePts in each trial selected above 
            ## (1 to 3 presentations of the same stimuli exist per session in Prabhat's data)
            numTimePtsPerTrial = np.empty((2,max(len(trials_stimA),len(trials_stimB))))
            numTimePtsPerTrial[:] = np.nan
            stimInd = 0;
            for thisStimTypeTrialNums in [trials_stimA, trials_stimB]:
                trialInd = 0
                for trial in thisStimTypeTrialNums:
                    inds_thisTrial = (df_animalSession['trial']==trial)
                    numTimePtsPerTrial[stimInd,trialInd] = np.sum(inds_thisTrial)
                    trialInd += 1
                stimInd += 1
            print(numTimePtsPerTrial) # rows are for stimuli type; cols are presentation of that stimulus            
            
            #### test and sort candidate comparisons based on whether the number of trials per session 
            ##      and approximate number of timePts match
            
            ## no trials of either type --> discard this comparison for this animal/session   
            if np.all(np.isnan(numTimePtsPerTrial)):
                print("discarded: neither stimulus type were found for this animal and session")
                break # skip to next session (WORK: handle this)
                
            ## different numbers of trials per stimuli/session --> discard this comparison for this animal/session 
            elif np.any(np.isnan(numTimePtsPerTrial)): 
                print("discarded: mismatching numbers of trials per stimulus type for this animal/session")
                break # skip to next session (WORK: handle this)
            
            ## FULFILLED here: condition that allows analysis to proceed to attempted data
            elif not np.any(np.isnan(numTimePtsPerTrial)): 
                print("trial numbers match")
            else:
                raise RuntimeError('unexpected trial comparison occurred')
            
            print("checking approx num of time points")
            
            #### discard this comparison for this animal/session if number of time points are too dissimilar
            minTPs = np.min(numTimePtsPerTrial) 
            maxTPs = np.max(numTimePtsPerTrial)
            meanTPs = np.mean(numTimePtsPerTrial)
            stdTPs = np.std(numTimePtsPerTrial)
            print('min', minTPs)
            print('max', maxTPs)
            print('std', stdTPs)
            print('mean',meanTPs)
            if (np.abs(minTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))) \
                or (np.abs(maxTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))): 
                print("discarded this comparison because variance in trial length is above the user's threshold")
                break # skip to next session (WORK: handle this)
            
            ########## data that makes it this far will be truncated to shortest trial prepared for svm
            
            ## truncate longer trials to shortest trial and save to new df
            truncLst = []
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                tmp_inds_trunc = (df_animalSession['trial']==trial) & (df_animalSession['timePt'] < minTPs)
                tmp_df_trunc = df_animalSession[tmp_inds_trunc]
                truncLst.append(tmp_df_trunc)
            df_trunc = pd.concat(truncLst)
                    
            ## concatenate same cells 
            NUM_ROWS_INITIALIZE = 15 # this value shouldn't matter since it should expand if needed 
            neuronArr_anmlSess = np.empty((NUM_ROWS_INITIALIZE,int(minTPs)))
            for data in [df_anmlSessStimA, df_anmlSessStimB]:
                trials = np.unique(data['trial'].tolist())
                for trial in trials:
                    print("trial: ",trial)
                    print("\n appending same neurons in trial: ", trial)
                    neurons = np.unique(data['neuronID'].tolist())
                    tmp_sameStimNeuronsArr = np.empty((np.sum(neuron),))
                    for neuron in neurons:
                        tmp_neuronInds = (data['trial']==trial) & (data['neuronID']==neuron)
                        tmp_neuronSeries = data.loc[tmp_neuronInds,'CaSignal']
                        tmp_neuronColVec = tmp_neuronSeries.as_matrix()
                        tmp_sameStimNeuronsArr 
#                         print(type(tmp_neuronDF))
                        #                         print('neuronDF',tmp_neuronDF)
                        #                         tmp_neuronTraceVec =
                        
                    
                    
                    
                    
                    
                    
                    
#                     tmp_trialInds = (data['trial']==trial) 
#                     print(np.sum(tmp_trialInds))
#                     tmp_mat = data[tmp_trialInds].as_matrix()
#                     print(tmp_mat)
#                     print(type(tmp_neuronConcatArr))
#                     np.append(tmp_neuronConcatArr,tmp_mat)
#                     print(tmp_neuronConcatArr)
                    
                
            
            totalNumComparisons += 1    
            print('\n')
        print('########\n')

print('total number of comparisons: ', totalNumComparisons)

animal:  1
session: 1
USS baseline
[ 5 10 16] [ 1  8 15]
[[ 26808.  26784.  26808.]
 [  8880.   8712.  35424.]]
trial numbers match
checking approx num of time points
min 8712.0
max 35424.0
std 9980.77592174
mean 22236.0
discarded this comparison because variance in trial length is above the user's threshold
########

animal:  3
session: 1
USS baseline
[ 4 11 17] [ 1  8 15]
[[ 38010.  33810.  33690.]
 [ 20220.  20460.  20340.]]
trial numbers match
checking approx num of time points
min 20220.0
max 38010.0
std 7550.14072717
mean 27755.0
trial:  4

 appending same neurons in trial:  4
trial:  11

 appending same neurons in trial:  11
trial:  17

 appending same neurons in trial:  17
trial:  1

 appending same neurons in trial:  1
trial:  8

 appending same neurons in trial:  8
trial:  15

 appending same neurons in trial:  15


session: 2
USS baseline
[ 6 12 16] [ 1  8 15]
[[ 32130.  31290.  37380.]
 [ 23835.  25620.  23870.]]
trial numbers match
checking approx num of time points
min 23

trial:  6

 appending same neurons in trial:  6
trial:  12

 appending same neurons in trial:  12
trial:  16

 appending same neurons in trial:  16
trial:  5

 appending same neurons in trial:  5
trial:  11

 appending same neurons in trial:  11
trial:  19

 appending same neurons in trial:  19


session: 3
USS male
[ 3 14 17] [ 7  9 18]
[[ 34646.  36958.  39304.]
 [ 38998.  34884.  35938.]]
trial numbers match
checking approx num of time points
min 34646.0
max 39304.0
std 1833.90003363
mean 36788.0
trial:  3

 appending same neurons in trial:  3
trial:  14

 appending same neurons in trial:  14
trial:  17

 appending same neurons in trial:  17
trial:  7

 appending same neurons in trial:  7
trial:  9

 appending same neurons in trial:  9
trial:  18

 appending same neurons in trial:  18


########

animal:  4
session: 1
USS male
[ 4 11] [ 6 14]
[[ 63672.  62328.]
 [ 62720.  62496.]]
trial numbers match
checking approx num of time points
min 62328.0
max 63672.0
std 520.076917388
mean 6

KeyboardInterrupt: 

In [68]:
np.sum(neurons)

1485

[ 6 10  5 15]


In [19]:
testInds = (df['animal']==1) & (df['session']==1) & (df['stimulus']=='USS') & (df['trial']==5)
print(np.sum(testInds))

26808


In [ ]:
np.unique(df_anmlSessStimA[df_anmlSessStimA['trial'] == trialA].tolist())

In [17]:

# gb = df.groupby(['stimulusType']).get_group('rat')

stimGrouped = df.groupby('stimulusType')
ratGrpd = stimGrouped.get_group('rat')

print(ratGrpd)

print('reached end of file')

gb = df.groupby('stimulusType')

df_means = gb.apply(np.mean)
df_means

for stimType in gb

               date  animalNum  sessionNum  trialNum stimulusType  neuronID  \
36240    2017_05_00          1           1         3          rat         1   
36241    2017_05_00          1           1         3          rat         1   
36242    2017_05_00          1           1         3          rat         1   
36243    2017_05_00          1           1         3          rat         1   
36244    2017_05_00          1           1         3          rat         1   
36245    2017_05_00          1           1         3          rat         1   
36246    2017_05_00          1           1         3          rat         1   
36247    2017_05_00          1           1         3          rat         1   
36248    2017_05_00          1           1         3          rat         1   
36249    2017_05_00          1           1         3          rat         1   
36250    2017_05_00          1           1         3          rat         1   
36251    2017_05_00          1           1         3

In [28]:
ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

print(ussData)

# slicing
df_big_force = df.loc[df['impact force (mN)'] > 1000, :]

df = pd.concat((df_low, df_high), axis=1)

# Specify indices we want (note parentheses holding each Boolean)
inds = (df['food density'] == 'high') & (df['cross-sectional area (sq micron)'] > 2000)

# Pull out areas
df.loc[inds, 'cross-sectional area (sq micron)']

df.corr()


# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})



# Write out DataFrame
df.to_csv('xa_combined.csv', index=False)

KeyError: 'stimulusType'